# Things I did in preprocessing
- Building the threads and a mechanic to query for users
- Build the context sphere by having all comments a user wrote + the full thread the user wrote it in
- Reduce the overall context length by excluding non relevant comments -> which are after a user comment written

In [1]:
# Merge csv Posting and Votes
import pandas as pd
from IPython.display import display, HTML

# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv(r'./data/raw_csv/Postings_01052019_31052019.csv')


# Display the first 10 rows
display(df.head(10))
print(df.to_html())

FileNotFoundError: [Errno 2] No such file or directory: './data/raw_csv/Postings_01052019_31052019.csv'

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pickle
import tiktoken

class DataPreprocessor:
    """
    A class to preprocess the raw comment data.
    """

    def __init__(self, file_path):
        """
        Initialize the DataPreprocessor with the path to the raw data file.
        
        :param file_path: str, path to the raw CSV file
        """
        self.file_path = file_path
        self.df = None

    def load_data(self):
        """Load the raw data from CSV file."""
        self.df = pd.read_csv(self.file_path)

    def preprocess(self):
        """Apply all preprocessing steps to the data."""
        self._convert_dates()
        self._handle_missing_values()
        self._convert_id_columns()
        self._create_new_features()

    def _convert_dates(self):
        """Convert date columns to datetime format."""
        date_columns = ['PostingCreatedAt', 'ArticlePublishingDate', 'UserCreatedAt']
        for col in date_columns:
            self.df[col] = pd.to_datetime(self.df[col])

    def _handle_missing_values(self):
        """Handle missing values in the dataset."""
        self.df['PostingHeadline'] = self.df['PostingHeadline'].fillna('No Headline')
        self.df['PostingComment'] = self.df['PostingComment'].fillna('No Comment')
        self.df['UserGender'] = self.df['UserGender'].fillna('Unknown')
        self.df['UserCommunityName'] = self.df['UserCommunityName'].fillna('Unknown')

    def _convert_id_columns(self):
        """Convert ID columns to integers."""
        id_columns = ['ID_Posting', 'ID_Posting_Parent', 'ID_CommunityIdentity', 'ID_Article']
        for col in id_columns:
            self.df[col] = self.df[col].fillna(0).astype(int)

    def _create_new_features(self):
        """Create new features from existing data."""
        self.df['CommentLength'] = self.df['PostingComment'].str.len()
        self.df['DaysSinceUserCreation'] = (self.df['PostingCreatedAt'] - self.df['UserCreatedAt']).dt.days
        self.df['IsReply'] = self.df['ID_Posting_Parent'] != 0
        self.df['PostingHour'] = self.df['PostingCreatedAt'].dt.hour
        self.df['PostingDayOfWeek'] = self.df['PostingCreatedAt'].dt.dayofweek

    def save_preprocessed_data(self, output_path):
        """
        Save the preprocessed data to a CSV file.
        
        :param output_path: str, path to save the preprocessed CSV file
        """
        self.df.to_csv(output_path, index=False)

        def save_preprocessed_data(self, output_path):
            """
            Save the preprocessed data to a pickle file.
            
            :param output_path: str, path to save the preprocessed pickle file
            """
        with open(output_path, 'wb') as f:
            pickle.dump(self.df, f)
        print(f"Preprocessed data saved to {output_path}")

    @classmethod
    def load_preprocessed_data(cls, input_path):
        """
        Load preprocessed data from a pickle file.
        
        :param input_path: str, path to the preprocessed pickle file
        :return: DataPreprocessor instance with loaded data
        """
        with open(input_path, 'rb') as f:
            df = pickle.load(f)

        preprocessor = cls(None)  # Create instance without file path
        preprocessor.df = df
        print(f"Preprocessed data loaded from {input_path}")
        return preprocessor

In [3]:
import json 

class CommentThreadManager:
    """
    A class to manage and structure comment threads for articles.
    """

    def __init__(self, df):
        """
        Initialize the CommentThreadManager with a preprocessed DataFrame.
        
        :param df: pandas DataFrame, preprocessed comment data
        """
        self.article_comments = {article_id: group for article_id, group in df.groupby('ID_Article')}

    def build_comment_thread(self, comments, parent_id, depth=0):
        thread = []
        replies = comments[comments['ID_Posting_Parent'] == parent_id]
        for _, reply in replies.iterrows():
            sub_thread = self.build_comment_thread(comments, int(reply['ID_Posting']), depth + 1)
            thread_item = {
                'id': int(reply['ID_Posting']),
                'parent_id': int(reply['ID_Posting_Parent']) if pd.notnull(reply['ID_Posting_Parent']) else None,
                'user_id': int(reply['ID_CommunityIdentity']),
                'user_name': reply['UserCommunityName'],
                'user_gender': reply['UserGender'],
                'user_created_at': reply['UserCreatedAt'].isoformat() if pd.notnull(reply['UserCreatedAt']) else None,
                'comment_headline': reply['PostingHeadline'],
                'comment_text': reply['PostingComment'],
                'comment_created_at': reply['PostingCreatedAt'].isoformat() if pd.notnull(reply['PostingCreatedAt']) else None,
                'comment_length': int(reply['CommentLength']),
                'depth': depth,
                'replies': sub_thread,
                'thread_stats': {
                    'total_comments': 1 + sum(r['thread_stats']['total_comments'] for r in sub_thread),
                    'max_depth': depth + max([r['thread_stats']['max_depth'] for r in sub_thread] + [0]),
                }
            }
            thread.append(thread_item)
        return thread

    def get_article_threads(self, article_id):
        """
        Get the structured comment threads for a specific article.
        
        :param article_id: int, ID of the article
        :return: dict, structured article data with comment threads
        """
        if article_id not in self.article_comments:
            return None

        article_df = self.article_comments[article_id]
        root_comments = article_df[article_df['ID_Posting_Parent'].isnull() | (article_df['ID_Posting_Parent'] == 0)]

        threads = []
        for _, comment in root_comments.iterrows():
            thread = {
                'id': int(comment['ID_Posting']),
                'parent_id': None,
                'user_id': int(comment['ID_CommunityIdentity']),
                'user_name': comment['UserCommunityName'],
                'user_gender': comment['UserGender'],
                'user_created_at': comment['UserCreatedAt'].isoformat() if pd.notnull(comment['UserCreatedAt']) else None,
                'comment_headline': comment['PostingHeadline'],
                'comment_text': comment['PostingComment'],
                'comment_created_at': comment['PostingCreatedAt'].isoformat() if pd.notnull(comment['PostingCreatedAt']) else None,
                'comment_length': int(comment['CommentLength']),
                'depth': 0,
                'replies': self.build_comment_thread(article_df, int(comment['ID_Posting']), 1)
            }
            threads.append(thread)

        article_meta = article_df.iloc[0]

        return {
            'article_id': int(article_id),
            'article_title': article_meta['ArticleTitle'],
            'article_publish_date': article_meta['ArticlePublishingDate'].isoformat() if pd.notnull(article_meta['ArticlePublishingDate']) else None,
            'article_channel': article_meta['ArticleChannel'],
            'article_ressort_name': article_meta['ArticleRessortName'],
            'total_comments': len(article_df),
            'root_comments': len(root_comments),
            'comment_threads': threads
        }

    def get_article_ids(self):
        """
        Get a list of all article IDs in the dataset.
        
        :return: list of int, article IDs
        """
        return list(self.article_comments.keys())

    def get_user_ids(self):
        """
        Get a list of all user IDs in the dataset.
        
        :return: list of int, user IDs
        """
        return self.df['ID_CommunityIdentity'].unique().tolist()

# Now lets start building the context sphere
I understand your point. You're looking to create a more comprehensive view of a user's activity and interactions within the comment threads. Instead of just seeing isolated comments made by a user, you want to see the full context of their engagement. This includes:

1. All comments made by the user across different articles.
2. For each comment, you want to see the entire thread it belongs to, not just the user's comment in isolation.
3. This will show what the user was responding to and how their comment fits into the larger conversation.
4. It will also reveal any subsequent responses to the user's comments.

The goal is to build a "context sphere" around each user, showing their complete interaction history within the comment ecosystem. This approach will provide a more nuanced understanding of the user's behavior, opinions, and how they engage with others in discussions.

Is this interpretation correct? If so, I can suggest how to modify your existing code to achieve this goal.

In [4]:
class UserContextSphere:
    def __init__(self, df, thread_manager):
        self.df = df
        self.thread_manager = thread_manager
        self.user_comments = {user_id: group for user_id, group in df.groupby('ID_CommunityIdentity')}

    def get_user_context(self, user_id):
        if user_id not in self.user_comments:
            return None

        user_df = self.user_comments[user_id]

        # Calculate user statistics
        total_comments = len(user_df)
        total_replies = len(user_df[user_df['ID_Posting_Parent'].notnull()])

        user_context = {
            'user_id': int(user_id),
            'user_name': user_df['UserCommunityName'].iloc[0],
            'user_gender': user_df['UserGender'].iloc[0],
            'user_created_at': user_df['UserCreatedAt'].iloc[0].isoformat(),
            'total_comments': total_comments,
            'total_replies': total_replies,
            'articles': {}
        }

        # Group comments by article
        for article_id, article_comments in user_df.groupby('ID_Article'):
            article_id = int(article_id)
            article_thread = self.thread_manager.get_article_threads(article_id)

            if article_thread:
                user_context['articles'][article_id] = {
                    'article_title': article_thread['article_title'],
                    'article_publish_date': article_thread['article_publish_date'],
                    'user_comments_count': len(article_comments),
                    'user_replies_count': len(article_comments[article_comments['ID_Posting_Parent'].notnull()]),
                    'threads': []
                }

                for _, comment in article_comments.iterrows():
                    thread = self.find_thread_for_comment(article_thread['comment_threads'], int(comment['ID_Posting']))
                    if thread:
                        user_context['articles'][article_id]['threads'].append(thread)

        return user_context

    def find_thread_for_comment(self, threads, comment_id):
        for thread in threads:
            if thread['id'] == comment_id:
                return thread
            if thread['replies']:
                result = self.find_thread_for_comment(thread['replies'], comment_id)
                if result:
                    return thread  # Return the whole thread, not just the subthread
        return None

    def count_tokens(self, text):
        encoding = tiktoken.get_encoding("cl100k_base")
        tokens_lst = encoding.encode(text)
        return len(tokens_lst)

    def generate_user_report_with_token_count(self, user_id):
        report = self.generate_user_report(user_id)
        token_count = self.count_tokens(report)
        report += f"\nTotal token count: {token_count}"
        return report, token_count

    def format_thread(self, thread, user_id, depth):
        indent = "│   " * depth
        is_user = thread['user_id'] == user_id
        user_indicator = "👤" if is_user else "  "
    
        formatted = f"{indent}{'└─ ' if depth > 0 else ''}{user_indicator} {thread['user_name']} "
        formatted += f"({thread['user_gender']}) - {thread['comment_created_at']}\n"
        formatted += f"{indent}{'   ' if depth > 0 else ''}  {thread['comment_text']}\n"
        formatted += f"{indent}{'   ' if depth > 0 else ''}  Length: {thread['comment_length']} chars\n"
    
        if thread['replies']:
            formatted += f"{indent}{'   ' if depth > 0 else ''}  Replies: {len(thread['replies'])}\n"
    
        formatted += "\n"
    
        for reply in thread['replies']:
            formatted += self.format_thread(reply, user_id, depth + 1)
    
        return formatted

    def generate_user_report(self, user_id):
        user_context = self.get_user_context(user_id)
        if not user_context:
            return f"No data found for user ID {user_id}"

        report = f"User Report for ID: {user_context['user_id']}\n"
        report += f"Name: {user_context['user_name']}\n"
        report += f"Gender: {user_context['user_gender']}\n"
        report += f"Created At: {user_context['user_created_at']}\n"
        report += f"Total Comments: {user_context['total_comments']}\n"
        report += f"Total Replies: {user_context['total_replies']}\n\n"

        for article_id, article_data in user_context['articles'].items():
            report += f"Article: {article_data['article_title']}\n"
            report += f"Published: {article_data['article_publish_date']}\n"
            report += f"User Comments on this Article: {article_data['user_comments_count']}\n"
            report += f"User Replies on this Article: {article_data['user_replies_count']}\n\n"

            report += "Comment Threads:\n"
            for thread in article_data['threads']:
                report += self.format_thread(thread, user_id, 0)
                report += "\n"

        return report

In [5]:
import os

if __name__ == "__main__":
    preprocessed_file = "data/preprocessed/preprocessed_data.pkl"

    if not os.path.exists(preprocessed_file):
        print("Preprocessed data not found. Preprocessing...")
        preprocessor = DataPreprocessor('./data/raw_csv/Postings_01052019_31052019.csv')
        preprocessor.load_data()
        preprocessor.preprocess()
        preprocessor.save_preprocessed_data(preprocessed_file)
    else:
        print("Loading preprocessed data...")
        preprocessor = DataPreprocessor.load_preprocessed_data(preprocessed_file)

    # Create CommentThreadManager
    thread_manager = CommentThreadManager(preprocessor.df)

    # Create UserContextSphere
    user_context_sphere = UserContextSphere(preprocessor.df, thread_manager)

    # Example: Generate report for a specific user
    user_id = 633859  # Replace with the desired user ID
    user_report, token_count = user_context_sphere.generate_user_report_with_token_count(user_id)


print(user_report)


Preprocessed data not found. Preprocessing...


FileNotFoundError: [Errno 2] No such file or directory: './data/raw_csv/Postings_01052019_31052019.csv'

In [8]:
import os

if __name__ == "__main__":
    preprocessed_file = "data/preprocessed/preprocessed_data.pkl"

    if not os.path.exists(preprocessed_file):
        print("Preprocessed data not found. Preprocessing...")
        preprocessor = DataPreprocessor('./data/raw_csv/Postings_01052019_31052019.csv')
        preprocessor.load_data()
        preprocessor.preprocess()
        preprocessor.save_preprocessed_data(preprocessed_file)
    else:
        print("Loading preprocessed data...")
        preprocessor = DataPreprocessor.load_preprocessed_data(preprocessed_file)

        thread_manager = CommentThreadManager(preprocessor.df)
        article_id = 12345  # Beispiel-Artikel-ID
        user_id = 67890     # Optional: Ziel-Benutzer-ID
        
        # Für alle Threads im Artikel
        full_thread_json = thread_manager.get_full_article_thread_json(article_id)
        
        # Nur für Threads, die den Ziel-Benutzer enthalten
        user_thread_json = thread_manager.get_full_article_thread_json(article_id, user_id)
        
        print(full_thread_json)  # oder user_thread_json



Loading preprocessed data...
Preprocessed data loaded from data/preprocessed/preprocessed_data.pkl
None


In [8]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
tokens_lst = encoding.encode("tiktoken is great!")
print(len(tokens_lst))


6


In [6]:
import os
import pandas as pd
import tiktoken
from collections import defaultdict
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

class UserAnalyzer:
    def __init__(self, df):
        self.df = df
        self.encoding = tiktoken.get_encoding("cl100k_base")

    def count_users(self):
        return self.df['ID_CommunityIdentity'].nunique()

    def count_tokens(self, text):
        tokens_lst = self.encoding.encode(text)
        return len(tokens_lst)

    def get_token_distribution(self):
        user_token_counts = defaultdict(int)

        for _, row in self.df.iterrows():
            user_id = row['ID_CommunityIdentity']
            comment = row['PostingComment']
            if isinstance(comment, str):
                user_token_counts[user_id] += self.count_tokens(comment)

        return dict(user_token_counts)

    def analyze_token_distribution(self):
        token_distribution = self.get_token_distribution()

        total_users = len(token_distribution)
        total_tokens = sum(token_distribution.values())
        avg_tokens = total_tokens / total_users if total_users > 0 else 0

        sorted_distribution = sorted(token_distribution.items(), key=lambda x: x[1], reverse=True)

        return {
            'total_users': total_users,
            'total_tokens': total_tokens,
            'avg_tokens_per_user': avg_tokens,
            'top_10_users': sorted_distribution[:10],
            'bottom_10_users': sorted_distribution[-10:],
        }

    def analyze_activity_patterns(self):
        self.df['PostingHour'] = self.df['PostingCreatedAt'].dt.hour
        self.df['PostingDayOfWeek'] = self.df['PostingCreatedAt'].dt.dayofweek

        hourly_activity = self.df['PostingHour'].value_counts().sort_index()
        daily_activity = self.df['PostingDayOfWeek'].value_counts().sort_index()

        return {
            'hourly_activity': hourly_activity,
            'daily_activity': daily_activity
        }

    def segment_users(self, n_clusters=3):
        user_activity = self.df.groupby('ID_CommunityIdentity').agg({
            'ID_Posting': 'count',
            'CommentLength': 'mean',
            'PostingHour': 'mean',
            'PostingDayOfWeek': 'mean'
        }).reset_index()

        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        user_activity['Cluster'] = kmeans.fit_predict(user_activity.drop('ID_CommunityIdentity', axis=1))

        return user_activity

    def visualize_activity_patterns(self):
        activity_patterns = self.analyze_activity_patterns()

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

        sns.barplot(x=activity_patterns['hourly_activity'].index,
                    y=activity_patterns['hourly_activity'].values, ax=ax1)
        ax1.set_title('Hourly Activity Pattern')
        ax1.set_xlabel('Hour of Day')
        ax1.set_ylabel('Number of Comments')

        sns.barplot(x=activity_patterns['daily_activity'].index,
                    y=activity_patterns['daily_activity'].values, ax=ax2)
        ax2.set_title('Daily Activity Pattern')
        ax2.set_xlabel('Day of Week (0 = Monday, 6 = Sunday)')
        ax2.set_ylabel('Number of Comments')

        plt.tight_layout()
        plt.savefig('activity_patterns.png')
        plt.close()

    def visualize_token_distribution(self):
        token_distribution = self.get_token_distribution()
        token_counts = list(token_distribution.values())

        plt.figure(figsize=(10, 6))
        sns.histplot(token_counts, kde=True)
        plt.title('Distribution of Token Counts per User')
        plt.xlabel('Token Count')
        plt.ylabel('Number of Users')
        plt.savefig('token_distribution.png')
        plt.close()

    def visualize_user_segments(self):
        user_segments = self.segment_users()

        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=user_segments, x='ID_Posting', y='CommentLength', hue='Cluster', palette='deep')
        plt.title('User Segments based on Activity and Comment Length')
        plt.xlabel('Number of Comments')
        plt.ylabel('Average Comment Length')
        plt.savefig('user_segments.png')
        plt.close()

def main_analysis():
    preprocessed_file = "data/preprocessed/preprocessed_data.pkl"

    if not os.path.exists(preprocessed_file):
        print("Preprocessed data not found. Preprocessing...")
        preprocessor = DataPreprocessor('./data/raw_csv/Postings_01052019_31052019.csv')
        preprocessor.load_data()
        preprocessor.preprocess()

        os.makedirs(os.path.dirname(preprocessed_file), exist_ok=True)
        with open(preprocessed_file, 'wb') as f:
            pickle.dump(preprocessor.df, f)

        df = preprocessor.df
    else:
        print("Loading preprocessed data...")
        with open(preprocessed_file, 'rb') as f:
            df = pickle.load(f)

    user_analyzer = UserAnalyzer(df)

    total_users = user_analyzer.count_users()
    print(f"Total number of unique users: {total_users}")

    print("Analyzing token distribution...")
    analysis_results = user_analyzer.analyze_token_distribution()

    print(f"Total users analyzed: {analysis_results['total_users']}")
    print(f"Total tokens across all users: {analysis_results['total_tokens']}")
    print(f"Average tokens per user: {analysis_results['avg_tokens_per_user']:.2f}")

    print("\nTop 10 users by token count:")
    for user_id, token_count in analysis_results['top_10_users']:
        print(f"User ID: {user_id}, Token count: {token_count}")

    print("\nBottom 10 users by token count:")
    for user_id, token_count in analysis_results['bottom_10_users']:
        print(f"User ID: {user_id}, Token count: {token_count}")

    print("\nAnalyzing activity patterns...")
    activity_patterns = user_analyzer.analyze_activity_patterns()
    print("Hourly activity pattern:", activity_patterns['hourly_activity'])
    print("Daily activity pattern:", activity_patterns['daily_activity'])

    print("\nSegmenting users...")
    user_segments = user_analyzer.segment_users()
    print(user_segments.groupby('Cluster').mean())

    print("\nGenerating visualizations...")
    user_analyzer.visualize_activity_patterns()
    user_analyzer.visualize_token_distribution()
    user_analyzer.visualize_user_segments()

if __name__ == "__main__":
    main_analysis()

Loading preprocessed data...
Total number of unique users: 23925
Analyzing token distribution...
Total users analyzed: 23925
Total tokens across all users: 38282469
Average tokens per user: 1600.10

Top 10 users by token count:
User ID: 499749, Token count: 104725
User ID: 518647, Token count: 91048
User ID: 601254, Token count: 79884
User ID: 521362, Token count: 75729
User ID: 678541, Token count: 72960
User ID: 1782, Token count: 70133
User ID: 59327, Token count: 69724
User ID: 665847, Token count: 67760
User ID: 686372, Token count: 66608
User ID: 633859, Token count: 65870

Bottom 10 users by token count:
User ID: 512297, Token count: 1
User ID: 565387, Token count: 1
User ID: 188861, Token count: 1
User ID: 669059, Token count: 1
User ID: 173522, Token count: 1
User ID: 537330, Token count: 1
User ID: 499935, Token count: 1
User ID: 694186, Token count: 1
User ID: 539016, Token count: 1
User ID: 670012, Token count: 1

Analyzing activity patterns...
Hourly activity pattern: Post